# Case Study 

 Data analysis into smart device data for a fitness device company's marketing enhancement.
 
 # How Can a Wellness Technology Company Play It Smart?
 
 ## Background
 
Founded in 2013, Bellabeat is a high-tech manufacturer of health-focused products, specially curated for women.

![](https://www.stableton.com/wp-content/webp-express/webp-images/uploads/2021/05/Bellabeat-Blog-Picture1.png.webp)
 
 ## Aim for Analysis- Business Task at hand
 
 - **Gain insight into how consumers are using their smart devices.**
 
 
- The analysis **will help guide future marketing strategies for Bellabeat products.**

## Stakeholders

We have to present the analysis to the **Bellabeat executive team along with our high-level recommendations for Bellabeat’s marketing strategy:**


•	*Urška Sršen*: Bellabeat’s cofounder and Chief Creative Officer


•	 ○ *Sando Mur*: Mathematician and Bellabeat’s cofounder; key member of the Bellabeat executive team 


•	○ *Bellabeat marketing analytics team*: A team of data analysts responsible for collecting, analyzing, and reporting data that helps guide Bellabeat’s marketing strategy.


## Bellabeat's current marketing strategy


- Bellabeat products became available through a growing number of **online retailers** in addition to their own **website.**
- The company has invested in **traditional advertising media: radio, out-of-home billboards, print, and television.**
- Focused on **digital marketing extensively:**.
- **invests year-round in Google Search** , maintaining active **Facebook and Instagram pages**, and consistently engages consumers on **twitter**.
- Runs video **ads on Youtube** and display **ads on the Google Display Network** to support campaigns around key marketing dates.

# 1) Ask Phase of Data Analysis : What do we need to know?

For effective results we need to ask the following questions and aim for their answers throughout our analysis:


1. What are some trends in smart device usage in non-Bellabeat customers?
2. How can these trends apply to Bellabeat customers for one of its product?
3. How can these trends help guide Bellabeat marketing strategy?

# 2) Prepare Phase of Data Analysis : What is our Data?

**Data Source:** 
[FitBit Fitness Tracker dataset](https://www.kaggle.com/datasets/arashnic/fitbit?datasetId=1041311&sortBy=voteCount)




**Data Collection:** 

**What?:** This Kaggle public data set contains personal fitness tracker data from thirty **FitBit** users, including minute-level data for physical activity, heart rate, and sleep monitoring.

**Who?:** The dataset was collected by *Amazon Mechanical Turk* from consenting FitBit users in their survey.

**Why?:** The data was to collected with an inspiration to understand Human temporal routine behavior and pattern recognition.




**Data Brief:**
The dataset contains 18 tables linked via the user IDs and timestamps. These tables contain information pertaining to the various intensities of physical activity,sedantary periods, calories burned (measured on daily basis, hourly basis and minute-wise basis)sleep duration(daily and minute-wise) and its frequency, weight logs, as well as heartrate.




**Data Limitations:**

* The data has been collected between 12-Mar-2016 and 12-May-2016. This makes it outdated for use in 2022.
* The sample size is of around 33 participants for most of the parameters and lesser for parmeters like heart rate per second (7) and weight (8). It is not sufficient to establish a good confidence level.
* The data set does not provide any demographic information pertaining to the participants which makes it difficult to analyse the data with respect to Bellabeat's target of female customers.




**Packages and Data Loading**

In [ ]:
#Loading the required Libraries

library(tidyverse)
library(skimr)
library(janitor)
library(dplyr)
library(lubridate)
library(ggplot2)
library(here)

In [ ]:
#Loading the Data

daily_activity <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv")
hourly_intensity<- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/hourlyIntensities_merged.csv")
daily_sleep<- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv")
weight<- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/weightLogInfo_merged.csv")

In [ ]:
#Preiviewing the first few rows of each table

head(daily_activity)
head(hourly_intensity)
head(daily_sleep)
head(weight)

# 3) Process Phase of Data Analysis: Is our Data clean and ready for Analysis?

For ensuring that each table is **tidy** we need to perform the following:

* Ensure the naming consistancy
* Check and Remove duplicates
* Make consistant date formats
* Store time and date in different columns

## Ensuring a naming consistancy and removing duplicates(if any)

In [ ]:

daily_activity<- daily_activity%>%
clean_names()%>%
unique()%>%
glimpse()
hourly_intensity<- hourly_intensity%>%
clean_names()%>%
unique()%>%
glimpse()
daily_sleep<-daily_sleep%>%
clean_names()%>%
unique()%>%
glimpse()
weight<- weight%>%
clean_names()%>%
unique()%>%
glimpse()

## Making consistant date formats and storing date and time in different columns

In [ ]:
# daily_activity table
daily_activity$date<-mdy(daily_activity$activity_date)
#adding a new column "day" for finding out the day-wise trends
daily_activity$day<- format(daily_activity$date, "%A")

daily_activity<- subset(daily_activity, select= -c(activity_date))
head(daily_activity)

# hourly_intensity table

hourly_intensity$date<- mdy_hms(hourly_intensity$activity_hour)
hourly_intensity$intensity_hour<-format(hourly_intensity$date,format= "%H:%M")
hourly_intensity$intensity_day<-format(hourly_intensity$date, format= "%A")
hourly_intensity<- subset(hourly_intensity, select= -activity_hour)
head(hourly_intensity)

#sleep log table
daily_sleep$date<- mdy_hms(daily_sleep$sleep_day)
daily_sleep$day<- format(daily_sleep$date, "%A")
# adding a column to determine the time participants lie awake in bed
new_sleep<-daily_sleep%>%
mutate(total_time_awake_in_bed= ( total_time_in_bed- total_minutes_asleep) )%>%
glimpse()
new_sleep<- subset(new_sleep, select= -sleep_day)
head(new_sleep)

#weight table
weight$date<- mdy_hms(weight$date)
weight$weight_day<-format(weight$date, "%A")
head(weight)

## Ensuring that the individual distances sums up to the total distance

In [ ]:
l<-daily_activity%>%
mutate(new_sum= light_active_distance+moderately_active_distance+very_active_distance)%>%
subset(select=c(id, total_distance,new_sum, light_active_distance,moderately_active_distance,very_active_distance))
head(l)

As evident from above:
- **the sum of induvidual distances(light_active_distance,moderately_active_distance,very_active_distance) does not add upto the total distance specified.** This makes any analysis based upon the sum of these vaues incorrect. Hence we will not use the sum of these value in any part of our analysis.
- Another noticable fact is that the **most users are majorly lightly active throughout the total distance they cover**. Hence, the need to promote some high intensity activites, so that the users may see evident changes.

Now we will be **merging** the cleaned daily_activity and daily_sleep tables for ease of analysis and visualisation

In [ ]:
#merging daily_activity and new_sleep tables on date, day and userIDs.
#Since daily activity table has more number of participants we will use"all.x=TRUE" 
#to ensure that all the non-matching cases of x are appended to the result as well.

daily<- merge(daily_activity, new_sleep, by= c("id", "date", "day"), all.x= TRUE)
daily<- distinct(daily)
glimpse(daily)

# 4) Analyse Phase of data analysis: What trends does our data reveal?

## Summary statistics of all our tables: To get brief insights of our data

In [ ]:
# daily table

summary(select(daily, total_steps, total_distance, sedentary_minutes, calories, total_minutes_asleep, total_time_in_bed,   total_time_awake_in_bed
))

In [ ]:
# hourly_intensity table
 summary(select(hourly_intensity, total_intensity))

From the above we can note that:


* The mean of **total_steps** for about 34 participants is below 8,000 which is not sufficient to see maximum health benefits. This shows that most users of fitness devices would require some motivation to increase their activity levels.
* The mean **sedantry minutes** are a staggering 991.2 minutes (16.5 hours). This metric gives us the insight that **most fitness device users are likely to be people who have long sitting hours and do a desk job**- a great idea of target group for bellabeat marketing.
* The analysis shows that an **average participant remains awake in bed for 21-30 minutes**, before they are finally able to sleep. 
* The mean **Intensity** per hour for the participants was merely 12 minutes.

 There are days for each participant when *no information has been collected*. Let us find out the total number of days for each participant when the *data was collected*.

In [ ]:
days<- daily%>%
group_by(id)%>%
summarise(number_of_days_info_collected= n_distinct(date))%>%
arrange(number_of_days_info_collected)

In [ ]:
days

As we see,** information was not collected for all the participants on all the days**. Why did the device did not collect information on few days? was the device not used on said dates? or perhaps the participants were involved in other activites like jump rope, swimming, bicycling etc that the device could not register?

## Finding out the target audience


 We can do this by analysing either the steps taken, the calories burned or the very_active_minutes against the days of the week.

In [ ]:
# by Calories burned
ta_calories<-daily%>%
group_by(day)%>%
summarise(sum_calories= sum(calories) )
ta_calories

In [ ]:
# By very_active_minutes
ta_sum_very_active_mins<-daily%>%
group_by(day)%>%
summarise(sum_very_active_mins= sum(very_active_minutes) )
ta_sum_very_active_mins

In [ ]:
# Plotting the results down for better understanding, with geom_histogram and some fancy esthetics
ggplot(daily,mapping=aes(x=day, weight=very_active_minutes, fill=day))+geom_histogram(stat="count", colour="white", alpha=0.4)+labs(y="sum_very_active_minutes")+theme(panel.background=element_blank(), axis.line=element_line(colour="black"))

Let us confirm our analysis once, with similar calculations and plotting for **total_steps taken on each day**

In [ ]:
# By total_steps taken on each day
ta_sum_steps<- daily%>%
group_by(day)%>%
summarise(sum_steps= sum(total_steps))
ta_sum_steps

In [ ]:
# Plotting the results with  geom_col 

plot<-ggplot(ta_sum_steps, mapping=aes(x= day, y=sum_steps, fill=day))+geom_col(colour="white", alpha= 0.4)
plot+theme(panel.background=element_blank(), axis.line=element_line(colour="black"))

from the above we achieve that:
* The most active days for participants are **Tuesdays and Wednesdays**
* The least active day for participants are **Sundays followed by Fridays**

So the **target audience** for Bellabeat is one that most likely:
- aims to relax and rest on the beginning and end of weekends(Fridays and Sundays) but
- follows a routine and remains active throughout the workdays(mon-thursdays) ,
- while being most energetic and active during the mid-week period(Tuesdays and Wednesdays)

In [ ]:
ta_very_active_dist_sum<-daily%>%
group_by(day)%>%
summarise(very_active_dist_sum= sum(very_active_distance))
ta_very_active_dist_sum

### Let us also find out the time of the day when the users are most active

In [ ]:
ta_hourly_intensity<- hourly_intensity%>%
group_by(intensity_hour)%>%
summarise(sum_total_intensity= sum(total_intensity))
ta_hourly_intensity

In [ ]:
# Visulaising the results

p<-ggplot(ta_hourly_intensity, aes(intensity_hour, sum_total_intensity, colour=intensity_hour, fill=intensity_hour ))+ geom_col(alpha=0.1)
p+labs(x="time of day", y="sum of intensity")+ theme(axis.text.x= element_text(angle=90),legend.position = "top", panel.background=element_blank(), axis.line=element_line(colour="black"))

In [ ]:
#doing the same but with ggplot(disclaimer: it is not appropriate with these variables)
ggplot(ta_hourly_intensity, aes(intensity_hour, sum_total_intensity, colour=intensity_hour) )+ geom_point( )+theme(axis.text.x= element_text(angle=90),legend.position = "top" )

From the above we can note that:

* The most active Time period for participants is around **5pm to 7pm**
* The least active timeframe for the participants is obviously the early morning hours of **2am to 4am** 


So **the target audience for Bellabeat** is one that is most likely:
*  full time workers who focus on their physical health after work hours are finished (5pm to 7pm)

## Further Visual Analysis

Understanding the **correlation between the calories burned and the total steps taken**

In [ ]:
cor(x=daily$total_steps, y=daily$calories, method="pearson")

In [ ]:
ggplot(daily, aes(x= total_steps, y=calories))+geom_point(colour="purple")+ geom_smooth(alpha=0.1, colour= "orange")+
labs(title = "Correlation between total steps and Calories burned")+theme(panel.grid.major=element_blank(), panel.grid.minor=element_blank())

The above analysis shows a slightly **positive relation between increase in daily number of steps and calories burned** i.e with every increase in the number of steps, the calories burned increases.

This information can be used to **market features that involve setting and attaining goals to burn more calories by increasing the daily steps.**

Understanding the **correlation between Calories burned and the sleeplessness period in bed**

In [ ]:
daily_without_na<-daily%>%
drop_na()
glimpse(daily_without_na)

In [ ]:

cor(x=daily_without_na$total_time_awake_in_bed, y=daily_without_na$calories, method="pearson")

In [ ]:
ggplot(daily_without_na, mapping=aes(total_time_awake_in_bed, calories))+geom_point(colour="purple")+facet_wrap(~total_sleep_records) +geom_smooth(colour="orange")+ 
theme(panel.background=element_blank(), axis.line=element_line(colour="black"))

The analysis is depicting **slightly negative correlation between energy expenditure throughout the day and sleeplessness while in bed**. This means that an increased period of activity is associated with less time spend in bed before finally sleeping.

**Let us also analyse the relationship between sedantry minutes and sleep duration**

In [ ]:
cor(x=daily_without_na$total_minutes_asleep, y=daily_without_na$sedentary_minutes, method="pearson")

In [ ]:
#plotting the same
ggplot(daily_without_na, mapping=aes(total_minutes_asleep, sedentary_minutes))+geom_point(colour="purple")+facet_wrap(~total_sleep_records) +geom_smooth(colour="orange")+
theme(panel.background=element_blank(), axis.line=element_line(colour="black"))

As seen in both of the above analysis:
* Having increased activity throughout the day is related to less awake time before sleeping.
* Likewise increased periods of inactivty(sedantry minutes) is associted with poor amount of sleep. 

These insights can be used to **market the Bellabeat for saliant feature of improved sleep with regular physical activity.**

# Share Phase of Data Analysis: what is our high level recommendations for enhancing Bellabeat's marketing strategies?

## Summary of our Target Audience:

Bellabeat's marketing team needs to focus their marketing towards the user segment that are:
- **working adults that mostly does a routine(9-5) desk jobs.** These users indulge in some light_activity to maintain their health but they definitely need some motivation to increase their activity levels to reap maximum health benefits.
- These users also **have a sleepless phase of 20-30 minutes in bed** before they are finally able sleep.
- They **remain active throughout the workdays(Mon-Thurs)** but considers relaxing on the weekends.

![](https://media.gettyimages.com/photos/the-stairway-to-success-picture-id531402681?s=612x612)

### Recommendations for the data 

- The **sample size of the data is too small**, it needs to be expanded to draw any strong conclusions. 
- The **number of participants should be equal for all the parameters**, lack f it presented many difficulties while analysis. Some parameters have really few participants(weight logs and heart rate per second) which makes them unsuitable for analysis.
- the data for the of values for VeryActiveDistance, ModeratelyActiveDistance and LightActiveDistance **needs to be reassesed** as their sum does not add upto the total distance for daily activity, this renders any analysis that can be performed on sum of these parameters as inaccurate.
- Also, some **information on the demographics of the users** such as gender, age, and height , would provide deep insights for developing strategies, keeping in mind Bellabeat's vision of products curation especially for women.

### Recommendations for the app
- Along with tracking the active movements, Bellabeat can have enhanced features such as a **water log** so that users can track their **hydration status** and maintain their overall health.
- The app can also include sublte features of **notifications or alarms for sleep schedules** or going early to bed to enhance the user's overall experience.
- Lastly, the users have a number of days when no activity has been logged. One possibilty is that the app did not register other forms of physical activity like biking, swimming, or playing a sport, muscle strengthning exercises etc. The app should **include metrics to register these activities** as well to provide a wholesome user experience.

# Act phase of Data Analysis: What can be Bellabeat's future marketing strategy?

* Most users struggle to remain highly active thrughout the day(Overall, the duration of "light active minutes" is much higher than "very active minutes"). This can be used to **market some high intensity, short duration workouts so  that the customers reap more health benefits.**
* There were very few users who logged in their weight details every day because it is a manual task. Bellabeat can utilize this information to **promote features like weight log notifications and even daily alarms,** reminding the users about the scheduled time for their daily physical activity.
* Since the app generates a lot of health data, this information can be leveraged to **develop and sell personalised goals and activity suggestions.**
* Finally, Many Participants experience a sleepless period of about 20-30 minutes in bed, Bellabeat can **develop and promote sleep assisting features** such as sleep inducing music, sleep journals, etc.

**Thats all from my analysis!**

Thank You all for going through my case study of a fitness tracker data.

Any suggestions/discussions for improvement and enhancement of this study would be much appreciated.